# SHANOIR api access

API documentation available at 

https://github.com/fli-iam/shanoir-ng/wiki 

https://github.com/fli-iam/shanoir-ng/wiki/REST-Tutorial 

In [15]:
import requests
import json
import os
import zipfile
from tqdm.notebook import tqdm

def get_token():
    r = requests.post('https://shanoir.irisa.fr/auth/realms/shanoir-ng/protocol/openid-connect/token', 
                  data = {'client_id':'shanoir-uploader', 
                          'grant_type':'password', 
                          'username':'LOGIN', 
                          'password':'PASSWORD', 
                          'scope':'openid'})

    answer = r.json()
    token = answer['access_token']
    return token

get_token()

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJNQXBrRkRMZ2VrS1cxTGI0em1LcDh1VGlDbElWNXZiOTdhckE3c25DbmFJIn0.eyJleHAiOjE2MjE1OTA2MDAsImlhdCI6MTYyMTU5MDMwMCwianRpIjoiNGYzNGZlNTMtNzVjZS00NWM2LWE1NGUtYWE3Y2Y2ODZlODZlIiwiaXNzIjoiaHR0cHM6Ly9zaGFub2lyLmlyaXNhLmZyL2F1dGgvcmVhbG1zL3NoYW5vaXItbmciLCJhdWQiOiJhY2NvdW50Iiwic3ViIjoiYzUwMWI1MmUtZDBlYy00ZTlkLWJhNGQtYWM4Mjc4YjMyODFmIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoic2hhbm9pci11cGxvYWRlciIsInNlc3Npb25fc3RhdGUiOiIzMDQxMzg4YS04MzM1LTQ2MzgtYmIwMi05YTI0MWJlOTAyNTciLCJhY3IiOiIxIiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbIm9mZmxpbmVfYWNjZXNzIiwiUk9MRV9FWFBFUlQiLCJ1bWFfYXV0aG9yaXphdGlvbiJdfSwicmVzb3VyY2VfYWNjZXNzIjp7ImFjY291bnQiOnsicm9sZXMiOlsibWFuYWdlLWFjY291bnQiLCJtYW5hZ2UtYWNjb3VudC1saW5rcyIsInZpZXctcHJvZmlsZSJdfX0sInNjb3BlIjoib3BlbmlkIGVtYWlsIHByb2ZpbGUiLCJlbWFpbF92ZXJpZmllZCI6dHJ1ZSwibmFtZSI6IkFsYmFuIEdhaWduYXJkIiwicHJlZmVycmVkX3VzZXJuYW1lIjoiYWdhaWduYXJkIiwiZ2l2ZW5fbmFtZSI6IkFsYmFuIiwiZmFtaWx5X25hbWUiOiJHYWlnbmFyZCIsInVzZXJJZCI6MjIwLCJlbWFpbCI6ImFsYmFuLmdhaWd

# Get Subjects 
Here we build two index to bridge inclusion identifiers with internal shanoir identifiers. <br>
The example below requests data for the study ICAN (study id : 124).

In [16]:
def get_all_subjects():
    api_url = f'https://shanoir.irisa.fr/shanoir-ng/studies/subjects/124/allSubjects'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(api_url, headers=h)
    json_res = r.json()
    subjects_shanoir_id = {}
    subjects_ican_id = {}
    for s in json_res:
        subjects_ican_id[str(s['id'])] = str(s['name'])
        subjects_shanoir_id[str(s['name'])] = str(s['id'])
    return subjects_shanoir_id, subjects_ican_id

subjects_shanoir_id, subjects_ican_id = get_all_subjects()

# Accessing and downloading Shanoir datasets

In [17]:
def get_all_subjects(study_id):
    api_url = f'https://shanoir.irisa.fr/shanoir-ng/studies/subjects/{study_id}/allSubjects'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(api_url, headers=h)
    json_res = r.json()
    subjects_shanoir_id = {}
    subjects_study_id = {}
    for s in json_res:
        subjects_study_id[str(s['id'])] = str(s['name'])
        subjects_shanoir_id[str(s['name'])] = str(s['id'])
    return subjects_shanoir_id, subjects_study_id

def get_all_subjectsId(study_id):
    api_url = f'https://shanoir.irisa.fr/shanoir-ng/studies/subjects/{study_id}/allSubjects'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(api_url, headers=h)
    json_res = r.json()
    res = []
    for subject in json_res:
      res.append(subject['id'])
    return res

def get_acquisitions(subject_id):
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/datasets/examinations/subject/{subject_id}/study/124'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(dataset_api_url, headers=h)
    #print(r.text)
    json_res = r.json()
    res = []
    for exam in json_res:
        #print(exam)
        for acquisition in exam['datasetAcquisitions']:
            res.append(acquisition['id'])
    return res

def get_acquisition(acquisition_id):
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/datasets/datasetacquisition/{acquisition_id}'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(dataset_api_url, headers=h)
    #print(r.text)

    acq = r.json()
    print(json.dumps(acq, indent=True))
    print(acq['protocol'])

def get_examinations(subject_id):
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/datasets/examinations/subjectid/{subject_id}'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(dataset_api_url, headers=h)

    exams = r.json()
    return(exams)
    #print(json.dumps(exams, indent=True))

def get_examinationsByStudy(study_id):
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/datasets/examinations/study/{study_id}'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(dataset_api_url, headers=h)

    exams = r.json()
    return(exams)
    #print(json.dumps(exams, indent=True))

def get_datasets(subject_id):
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/datasets/datasets/subject/{subject_id}/study/124'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(dataset_api_url, headers=h)
    datasets = r.json()
    res = []
    for d in datasets:
        res.append(d)
    return(res)


def get_centerNames():
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/studies/studies/namesAndCenters'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(dataset_api_url, headers=h)
    studies = r.json()
    # Suppression des redondances entre les centres ican et ucan
    unique_center_names = set()

    for study in studies:
        for study_center in study.get("studyCenterList", []):
            center_name = study_center.get("center", {}).get("name")
            if center_name:
                unique_center_names.add(center_name)

    return list(unique_center_names)


def get_center(center_id):
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/studies/centers/{center_id}'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(dataset_api_url, headers=h)

    center = r.json()
    return(center)


def get_dataset(dataset_id):
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/datasets/datasets/{dataset_id}'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(dataset_api_url, headers=h)
    #print(r.text)

    dataset = r.json()
    return(dataset)
#    print(json.dumps(dataset, indent=True))

def get_studyStatistics(study_id):
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/studies/studies/statistics/{study_id}'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(dataset_api_url, headers=h)

    dataset = r.json()
    return(dataset)


def get_dicomMetadata(dataset_id):
    dataset_api_url = f'https://shanoir-qualif.irisa.fr/shanoir-ng/datasets/datasets/dicom-metadata/{dataset_id}'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(dataset_api_url, headers=h)
    if r.status_code == 200:
      dicom_metadata = r.json()
      return(dicom_metadata)
    elif r.status_code == 500:
        print("Internal Server Error (500)")
        return(None)


def download_dataset(subject_name, dataset_id):
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/datasets/datasets/download/{dataset_id}'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(dataset_api_url, headers=h)

    #print(r.status_code)
    #print(r.headers['content-type'])
    #print(r.encoding)

    if (r.status_code == 200):
        path = f'./{subject_name}/{dataset_id}'
        os.makedirs(path, exist_ok=True)
        with open(path+'/file.zip', 'wb') as f:
            f.write(r.content)
        with zipfile.ZipFile(path+'/file.zip','r') as zip_ref:
            zip_ref.extractall(path)
        os.remove(path+'/file.zip')
        print(f'successfully downloaded dataset to {path}')
    else :
        print(r.text)

def download_datasets(filepath, datasets_ids):
    dataset_api_url = f'https://shanoir-qualif.irisa.fr/shanoir-ng/datasets/datasets/massiveDownload/{datasets_ids}'
    h = {"Accept": "application/json", 'Authorization': 'Bearer '+get_token()}
    r = requests.get(dataset_api_url, headers=h)

    if (r.status_code == 200):
        path = f'{filepath}'
        #os.makedirs(path, exist_ok=True)
        with open(path+'/file.zip', 'wb') as f:
            f.write(r.content)
        with zipfile.ZipFile(path+'/file.zip','r') as zip_ref:
            zip_ref.extractall(path)
        os.remove(path+'/file.zip')
        print(f'successfully downloaded datasets to {path}')
    else :
        print(r.text)

def get_studyCenters(study_id):
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/studies/centers/names/{study_id}'
    h = {"Accept": "application/json", 'Authorization': 'Bearer ' + get_token()}
    r = requests.get(dataset_api_url, headers=h)

    return r.json()

def get_subjectsNames():
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/studies/subjects/names'
    h = {"Accept": "application/json", 'Authorization': 'Bearer ' + get_token()}
    r = requests.get(dataset_api_url, headers=h)

    return r.json()

def get_subject(subject_id):
    dataset_api_url = f'https://shanoir.irisa.fr/shanoir-ng/studies/subjects/{subject_id}'
    h = {"Accept": "application/json", 'Authorization': 'Bearer ' + get_token()}
    r = requests.get(dataset_api_url, headers=h)

    return r.json()

In [18]:
list_of_ids = ['AIC_25_0118',
'AIC_01_0502']

In [19]:
from tqdm.notebook import tqdm

for inclusion_id in tqdm(list_of_ids):
    print('--------')
    print(inclusion_id)
    if inclusion_id in subjects_shanoir_id.keys():
        print(subjects_shanoir_id[inclusion_id])
        for dataset_id in get_datasets(subjects_shanoir_id[inclusion_id]):
            dataset = get_dataset(dataset_id)
            #print(dataset)
            print(f'{dataset_id} : {dataset["name"]}')    
            if ('tof' in dataset['name'].lower()) or ('angio' in dataset['name'].lower()):
                  print(f'TOF image available for download')

--------
AIC_25_0118
9677
379659 : T2 TSE AX
379660 : TOF 3D WILLIS
TOF image available for download
--------
AIC_01_0502
9425
375223 : 375223 2012-10-29 MR



In [ ]:
### RETRIEVING INFORMATIONS FROM A SHANOIR SUBJECT IDS LIST ###

import pandas as pd
from IPython.display import FileLink

## OBJECTIVE : RESEARCHER WITH A LIST OF SUBJECT IDS WANTS TO RETRIEVE THE FOLLOWING FIELDS ASSOCIATED WITH SHANOIR IDS: 
# Name of the study
# Name of the center where TOF exam occured
# Date of TOF exam

# Creation of a subject's names list from original file (with 1 subject name per line without header)
with open('{FILEPATH}', 'r') as file:
    namesList = [line.strip() for line in file]

print(namesList)

# Getting Shanoir's subjects list
subjects_df = pd.DataFrame(get_subjectsNames())
name_to_id = {row['name']: row['id'] for index, row in subjects_df.iterrows()}

# Creation of results dataframe
results = pd.DataFrame(columns=['subject_name', 'study_name', 'center_name', 'exam_date'])

# For each subject name of the original list, if this name matches a Shanoir subject id , we retrieve informations linked to this subject
for subject_name in namesList :
  if subject_name in name_to_id :
    # Getting subject id linked to subject name
    subject_id = name_to_id[subject_name]
    # Getting informations from subject
    subject_info = get_subject(subject_id)
    # Getting study linked to the subject
    study_name = subject_info["subjectStudyList"][0]["study"]["name"]
    # Getting exams from the subject
    exams_data = get_examinations(subject_id)
    exams_info = pd.DataFrame(exams_data)

    # Converting column 'examinationDate' in datetime if not already
    exams_info['examinationDate'] = pd.to_datetime(exams_info['examinationDate'])
    # Find oldest exam date
    index_oldest_exam = exams_info['examinationDate'].idxmin()
    # Getting oldest exam
    oldest_exam = exams_info.loc[index_oldest_exam]

    center_id = None
    exam_date = None

    # Getting infos from oldest TOF exam
    for row_index, row in exams_info.iterrows() :
      if ('tof' in row['comment'].lower()) or ('angio' in row['comment'].lower()) or ('time of flight' in row['comment'].lower()):
        center_id = row['centerId']
        exam_date = row['examinationDate']
        break
    if center_id is None:
      center_id = oldest_exam['centerId']
      exam_date = oldest_exam['examinationDate']
    
    # Getting center name from center id
    center_info = get_center(center_id)
    center_name = center_info['name']

    #results = results.append({'subject_name': subject_name, 'study_name': study_name, 'center_name': center_name, 'exam_date': exam_date}, ignore_index=True)
    new_row = {'subject_name': subject_name, 'study_name': study_name, 'center_name': center_name, 'exam_date': exam_date}

    results = pd.concat([results, pd.DataFrame([new_row])], ignore_index=True)

results_str = results.to_string(index=False)
print(results_str)

# Saving as CSV
#results.to_csv('results.csv', index=False)

# Creating download link to csv file
#FileLink('results.csv')

In [ ]:
# Script for datasets export

## OBJECTIVE
Getting a list of dataset ids from different studies based on specific DICOM metadata values

## RULES
Images from different centers <br>
+ At least 50 images in the batch of the sequence <br>
+ AND slice thickness < 0.5 cm <br>
+ AND terms: time of flight OR TOF or MRA or angiography <br>

## METADATA
ProtocolName 0018,1030 <br>
SeriesDescription 0008,103E <br>
SliceThickness 0018,0050 <br>
NumberOfSlices 2001,1018 <br>
NumberOfFrames 0028,0008 <br>
DataElements 07A1,1002 <br>

In [ ]:
import pandas as pd

# define number of datasets desired
dataset_size = 10
# define studies of interest 124 : ICAN, 178 : UCAN
studies = {'124','178'}

# filepath where datasets are downloaded
filepath = ''

all_subjectsId = []
all_datasetsId = []
results_datasetsId = []
mri_types = ["tof","angio","angiography","time of flight","mra"]


# GET ALL SUBJECTS OF STUDIES OF INTEREST
for study in studies :
  count = 0
  # get a list of all subjects id for this study
  all_subjectsId = get_all_subjectsId(study)
  for subject in all_subjectsId :
    # get a list of all datasets id from this subject id and this study id
    all_datasetsId = get_datasets(subject, study)
    for dataset in all_datasetsId :
      is_tof = False
      enough_frames = False
      thin_enough = False
      # get dicom metadata for each dataset and check quality compliance
      metadata = get_dicomMetadata(dataset)
      if metadata is not None :
        for item in metadata:
          # Check if ProtocolName or SeriesDescription contains "tof" or "angio" or "flight"
          if ('0008103E' in item and any(x in item['0008103E']["Value"][0].lower() for x in mri_types)) or ('00181030' in item and any(x in item['00181030']["Value"][0].lower() for x in mri_types)):
            is_tof = True

          if "00180050" in item and item["00180050"]["Value"] != [] and float(item["00180050"]["Value"][0]) < 5:
            thin_enough = True

          if ("20011018" in item and item["20011018"]["Value"] != [] and int(item["20011018"]["Value"][0]) > 50) or ("00280008" in item and item["00280008"]["Value"] != [] and int(item["00280008"]["Value"][0]) > 50) or ("07A11002" in item and item["07A11002"]["Value"] != [] and int(item["07A11002"]["Value"][0]) > 50):
            enough_frames = True

        if is_tof and thin_enough and enough_frames :
          # Avoid duplicates in case subjects in multiple studies
          if dataset not in results_datasetsId :
            results_datasetsId.append(dataset)
            count += 1
          if count == dataset_size/(len(studies)) :
            break
    if count == dataset_size/(len(studies)) :
      break

print(results_datasetsId)

# download_datasets(filepath, results_datasetsId)